### bbox

In [ ]:
import os
import json
from tqdm import tqdm


INPUT_DIR = "./dexart/results_bbox/bucket"


OUTPUT_PATH = f"{INPUT_DIR}/bboxes.json"


TARGET_NAME = INPUT_DIR.split('/')[-1]

bboxes_all = {}


for episode_folder in tqdm(sorted(os.listdir(INPUT_DIR)), desc="Processing episodes"):
    episode_path = os.path.join(INPUT_DIR, episode_folder)
    video_boxes_path = os.path.join(episode_path, "video_boxes.json")

    if not os.path.isfile(video_boxes_path):
        print(f"Missing video_boxes.json in {episode_folder}, skipped.")
        continue

    with open(video_boxes_path, "r") as f:
        frame_box_data = json.load(f)

    frame_keys = sorted(frame_box_data.keys(), key=lambda x: int(''.join(filter(str.isdigit, x))))
    episode_boxes = []

    for frame_key in frame_keys:
        objects = frame_box_data.get(frame_key, [])
        frame_boxes = []

        for obj in objects:
            if isinstance(obj, list) and len(obj) == 2:
                original_name, bbox = obj
                frame_boxes.append([TARGET_NAME, bbox])  

        episode_boxes.append([frame_boxes[0]] if frame_boxes else [])  


    episode_id = episode_folder.replace("episode_", "").replace("_combined", "")
    bboxes_all[episode_id] = episode_boxes


os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)


with open(OUTPUT_PATH, "w") as f:
    json.dump(bboxes_all, f)

print(f"Saved bboxes.json to {OUTPUT_PATH}")
